First goal is to find an endpoint for Form 4's

https://www.sec.gov/developer

https://www.sec.gov/os/accessing-edgar-data

In [60]:
import requests
import time
import datetime
import pandas as pd
import os

run_once_override = False

In [50]:
HEADERS = {
    'User-Agent': 'Personal User khalidelassaad@gmail.com',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'www.sec.gov'
}

def SEC_API_sleep():
    # Be a good citizen! Limit requests to less than 10 per second
    time.sleep(0.11)
    

Fetch the list of CIKs from here https://www.sec.gov/Archives/edgar/cik-lookup-data.txt

In [53]:
def get_url_to_outfilename(url, outfilename, run_once_override=run_once_override):
    if not run_once_override:
        return -1
    response = requests.get(url, headers=HEADERS)
    print("Status Code:", response.status_code)
    with open(outfilename, "w", encoding="utf-8") as outfile:
        outfile.write(response.text)
    SEC_API_sleep()
    return response

In [4]:
get_url_to_outfilename(
    "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt",
    "cik_list.txt"
)

-1

In [5]:
get_url_to_outfilename(
    "https://www.sec.gov/files/company_tickers.json",
    "company_tickers.json"
)

-1

In [6]:
def generate_interested_companies_dict():
    interested_company_names = []
    return_dict = dict()
    with open("interested_company_names.txt","r") as infile:
        for line in infile:
            interested_company_names.append(line.strip())
    with open("cik_list.txt","r") as infile:
        for line in infile:
            for name in interested_company_names:
                if name in line:
                    cik_number = int(line.split(":")[1])
                    cik_name = line.split(":")[0]
                    return_dict[cik_number] = {"name": cik_name}
    return return_dict

In [7]:
interested_companies_dict = generate_interested_companies_dict()

We can get information about a specific CIK's filings by querying this resource:

https://data.sec.gov/submissions/CIK##########.json

In [52]:
def save_interested_companies_indices(interested_companies_dict, run_once_override=run_once_override):
    if not run_once_override:
        return -1
    for cik in interested_companies_dict:
        url = "https://data.sec.gov/submissions/CIK{:010d}.json".format(cik)
        get_url_to_outfilename(
            "https://data.sec.gov/submissions/CIK{:010d}.json".format(cik),
            "{}.html".format(cik),
            run_once_override=run_once_override
        )
    return 0

The above URL is not fetching. Let's try something else.

https://www.sec.gov/os/accessing-edgar-data

According to the above, use https://www.sec.gov/Archives/edgar/full-index/ to get an index per quarter of all filings from all companies. Let's write a function to grab the latest index and parse out relevant links for our `interested_companies_dict`

We want the `master.idx` file which contains data in this format: `CIK|Company Name|Form Type|Date Filed|Filename`

The full path to this file looks like:

`https://www.sec.gov/Archives/edgar/full-index/{YYYY}/QTR{1-4}/master.idx`

Data extends from 1993 QTR1 - 2023 QTR1

In [24]:
def get_master_index_for_year_and_quarter(year, quarter, run_once_override=run_once_override):
    if not run_once_override:
        return -1
    get_url_to_outfilename(
        "https://www.sec.gov/Archives/edgar/full-index/{}/QTR{}/master.idx".format(year, quarter),
        "indices/{}Q{}_master.idx".format(year, quarter),
        run_once_override=run_once_override
    )
    return 0

In [35]:
get_master_index_for_year_and_quarter(2023, 1)

-1

Let's grab all indices since 1993 QTR1, then we can aggregate them together for one unified master index that we can parse for companies we're interested in.

First we'll grab all indices.

In [54]:
def get_all_master_indices(run_once_override=run_once_override):
    if not run_once_override:
        return -1
    start_year = 1993
    end_year = pd.Timestamp(datetime.datetime.now()).year
    end_quarter = pd.Timestamp(datetime.datetime.now()).quarter
    for year in range(start_year, end_year + 1):
        end_quarter_range = 4 if year != end_year else end_quarter
        for quarter in range(1, end_quarter_range + 1):
            print("Fetchin index for {}Q{}".format(year, quarter))
            get_master_index_for_year_and_quarter(year, quarter, run_once_override)
    return 0

In [56]:
get_all_master_indices()

-1

Let's also write a function to fetch only the latest index (for updates while the current quarter is ongoing).

In [57]:
def get_latest_master_index(run_once_override=run_once_override):
    if not run_once_override:
        return -1
    year = pd.Timestamp(datetime.datetime.now()).year
    quarter = pd.Timestamp(datetime.datetime.now()).quarter
    print("Fetchin index for {}Q{}".format(year, quarter))
    get_master_index_for_year_and_quarter(year, quarter, run_once_override)
    return 0

In [59]:
get_latest_master_index()

-1

Nice, now to read data. Here's what I want to do:

1. Read all index files.
2. Grab rows for CIKs that I'm interested in, based on `interested_companies_dict`
3. Output rows as a pandas dataframe

In [65]:
index_files = ["indices/" + filepath for filepath in os.listdir("indices")]
index_files[-1]

'indices/2023Q1_master.idx'

In [88]:
index_column_names = ["CIK","Company Name","Form Type","Date Filed","Filename"]
index_dataframe = pd.read_csv(index_files[-1], sep = "|", skiprows=11, names=index_column_names)
index_dataframe

,CIK,Company Name,Form Type,Date Filed,Filename
0,1000045,NICHOLAS FINANCIAL INC,10-Q,2023-02-14,edgar/data/1000045/0000950170-23-002704.txt
1,1000045,NICHOLAS FINANCIAL INC,4,2023-02-24,edgar/data/1000045/0001000045-23-000001.txt
2,1000045,NICHOLAS FINANCIAL INC,4,2023-02-28,edgar/data/1000045/0001000045-23-000002.txt
3,1000045,NICHOLAS FINANCIAL INC,4,2023-03-09,edgar/data/1000045/0001398344-23-005935.txt
4,1000045,NICHOLAS FINANCIAL INC,8-K,2023-01-24,edgar/data/1000045/0000950170-23-001040.txt
...,...,...,...,...,...
304649,9984,BARNES GROUP INC,SC 13G/A,2023-02-09,edgar/data/9984/0001104659-23-015428.txt
304650,9984,BARNES GROUP INC,SC 13G/A,2023-02-10,edgar/data/9984/0000354204-23-001504.txt
304651,9984,BARNES GROUP INC,SC 13G/A,2023-02-13,edgar/data/9984/0001315478-23-000022.txt
304652,9984,BARNES GROUP INC,SC 13G/A,2023-02-14,edgar/data/9984/0000070858-23-000070.txt


In [105]:
index_dataframe[index_dataframe["CIK"].isin(interested_companies_dict)]

,CIK,Company Name,Form Type,Date Filed,Filename
23906,1108524,"Salesforce, Inc.",10-K,2023-03-08,edgar/data/1108524/0001108524-23-000011.txt
23907,1108524,"Salesforce, Inc.",13F-HR,2023-02-10,edgar/data/1108524/0001108524-23-000005.txt
23908,1108524,"Salesforce, Inc.",3,2023-03-03,edgar/data/1108524/0001127602-23-008948.txt
23909,1108524,"Salesforce, Inc.",3,2023-03-03,edgar/data/1108524/0001127602-23-008950.txt
23910,1108524,"Salesforce, Inc.",3,2023-03-03,edgar/data/1108524/0001418812-23-000017.txt
...,...,...,...,...,...
23995,1108524,"Salesforce, Inc.",8-K,2023-01-27,edgar/data/1108524/0001193125-23-017661.txt
23996,1108524,"Salesforce, Inc.",8-K,2023-03-01,edgar/data/1108524/0001108524-23-000007.txt
23997,1108524,"Salesforce, Inc.",SC 13G/A,2023-02-07,edgar/data/1108524/0001306550-23-008329.txt
23998,1108524,"Salesforce, Inc.",SC 13G/A,2023-02-09,edgar/data/1108524/0000315066-23-002031.txt


Okay, I have the pieces to read the data correctly from the files and to focus on only the companies I care about. The last step is figure out how to concatenate two dataframes together.

Looks like `pd.concat([list of dataframes])` will do the job. Time to stitch it all together.